# FIT5196 Task 1 in Assessment 1
#### Student Name: Sarath Gopinathan
#### Student ID: 30434904

Date: 05/09/2020

Version: 1.4

Environment: Python 3.8.5 and Jupyter notebook

Libraries used: 
* re ( For regular expression, included in Anaconda Python 3.8.5) 
* OS ( For reading the files from Folders.)
* Langid ( For classification of English Tweets from others.)

### Table of Contents

* [Reading data](#reading_data)
* [Filtering Tweets](#lang_tweets)
* [Unique Tweets](#unique_tweets)
* [Formatting Tweets](#formatting_tweets)
* [Writing to file](#writing)

    

In [ ]:
import os
import re
import langid

### Reading data <a class="anchor" id="reading_data"></a>

In [2]:
%%time
path = "./30434904/"
dirs = os.listdir( path )

all_tweets_dict = []
        
# regex    
pattern_text = re.compile(r'(?<=("text":"))(.*?)(?=(","created_at")|(","id")|(","withheld")|("}],)|("},))')

pattern_created_at = re.compile(r'(?<=("created_at":"))(.*?)(?=(","text")|(","id")|(","withheld")|("}],)|("},))')

pattern_id = re.compile(r'(?<=("id":"))(.*?)(?=(","text")|(","created_at")|(","withheld")|("}],)|("},))')

# reading   
for i in range(len(dirs)):
    
    text_list = []
    created_at_list = []
    id_list = []
    
    with open(path+dirs[i],'r',encoding="utf-8") as infile:
        file = infile.read()
        
    matches_text = pattern_text.finditer(file)

    for match in matches_text:
        text_list.append(match.group(2))
    
    matches_created_at = pattern_created_at.finditer(file)

    for match in matches_created_at:
        created_at_list.append(match.group(2)[0:10])
    
    matches_id = pattern_id.finditer(file)

    # appending to a list 
    for match in matches_id:
        id_list.append(match.group(2))
        
    # appending to a list     
    for j in range(len(text_list)):
        
        input_values = {"date": created_at_list[j], "id": id_list[j], "tweet": text_list[j]}
        
        # appending to required list 
        all_tweets_dict.append(input_values)

Wall time: 41.8 s


### Filtering Tweets(Language) <a class="anchor" id="lang_tweets"></a>

In [3]:
%%time
filtered_tweets = []

for k in range(len(all_tweets_dict)):
    
        current_tweet = all_tweets_dict[k].get('tweet')
        
        tw = eval("\""+current_tweet+"\"").encode("utf-16","surrogatepass").decode("utf-16")
        
        tweet_language = langid.classify(tw)
        
        # filtering out all eng tweets
        if(tweet_language[0] == 'en'):
            
            input_unique_values = {"date": all_tweets_dict[k].get('date'), "id": all_tweets_dict[k].get('id'),
                               "tweet": current_tweet}
        
            filtered_tweets.append(input_unique_values)  

Wall time: 14min 50s


### Unique Tweets(Language) <a class="anchor" id="unique_tweets"></a>

In [4]:
%%time
# filtering out all unique tweets
unique_tweets = [filtered_tweets[0]]
for unique in filtered_tweets:
    if unique not in unique_tweets:
           unique_tweets.append(unique)

Wall time: 7min 22s


### Formatting the Tweets(Language) <a class="anchor" id="formatting_tweets"></a>

In [5]:
%%time
date_wise_tweets = []

for k in range(len(unique_tweets)):
    
    flag = 0
    
    for l in range(len(date_wise_tweets)):
    
        if(date_wise_tweets[l].get('date') == unique_tweets[k].get('date')):
            
            current_tweet = unique_tweets[k].get('tweet')
        
            tw = eval("\""+current_tweet+"\"").encode("utf-16","surrogatepass").decode("utf-16")
            
            # converting to xml format
            tw = str(tw).replace("<", "&lt;")
        
            tw = str(tw).replace(">", "&gt;")
        
            tw = str(tw).replace("&", "&amp;")
        
            tw = str(tw).replace("'", "&apos;")
        
            tw = str(tw).replace('"', "&quot;")
            
            insert_dict = {"id": unique_tweets[k].get('id'), "tweet": tw}
            
            date_wise_tweets[l].get('data').append(insert_dict)
            
            flag = 1            
        
    if(flag == 0):
        data = []
        
        current_tweet = unique_tweets[k].get('tweet')
        
        tw = eval("\""+current_tweet+"\"").encode("utf-16","surrogatepass").decode("utf-16")
        
        # converting to xml format
        tw = str(tw).replace("<", "&lt;")
        
        tw = str(tw).replace(">", "&gt;")
        
        tw = str(tw).replace("&", "&amp;")
        
        tw = str(tw).replace("'", "&apos;")
        
        tw = str(tw).replace('"', "&quot;")

        insert_dict = {"id": unique_tweets[k].get('id'), "tweet": tw}

        data.append(insert_dict)

        date_insert_dict = {"date": unique_tweets[k].get('date'), "data": data}
        
        # appending to final list (required format)
        date_wise_tweets.append(date_insert_dict) 

Wall time: 2.42 s


### Writing to file <a class="anchor" id="writing"></a>

In [6]:
%%time
# writing to file
with open("./30434904.xml", "w", encoding="utf-8") as file:
    file.write("<?xml version=\"1.0\" encoding=\"UTF-8\"?>\n<data>\n")
    
    for i in range (len(date_wise_tweets)):
        
        file.write("<tweets date=\"" + date_wise_tweets[i].get('date') + "\">\n")
        
        for j in range(len(date_wise_tweets[i].get('data'))):
            
            file.write("<tweet id=\"" + date_wise_tweets[i].get('data')[j].get('id') + "\">" + 
               date_wise_tweets[i].get('data')[j].get('tweet') + "</tweet>\n")
            
        file.write("</tweets>\n")
    
    file.write("</data>")


Wall time: 252 ms
